In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import glob
from datasets import load_dataset
import datasets

In [6]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [7]:
import pandas as pd

df = pd.read_csv("preproses_lower.csv")
pd.set_option('display.max_colwidth', None)
print(df.dtypes)
df

INISIAL             object
RESPON              object
LEVEL KOMPETENSI     int64
English             object
Summaries           object
dtype: object


,INISIAL,RESPON,LEVEL KOMPETENSI,English,Summaries
0,AM,Konflik kepentingan muncul pada saat terjadinya pertentangan antara pelaksanaan tugas dan kepentingan pribadi yang mempengaruhi keputusan yang akan dibuat dan jika hal ini tidak ditangani dengan baik dapat berujung pelanggaran etika administratif bahkan pidana Oleh karena itu dalam menghadapi terjadinya konflik kepentingan saat ini hal yang dilakukan adalah berusaha untuk mencegah dengan menghindari atau membatasi sumber sumber konflik kepentingan diantaranya o Menghindari melakukan pekerjaan di luar pekerjaan saat ini o Membatasi atau menghindari kepemilikan aset pada perusahaan yang dapat terkait dengan pelaksanaan tugas jabatan o Menghindari dan membatasi diri berinteraksi langsung dengan pihak pihak terkait yang dilarang oleh kode etik dan perundangan atau berpotensi dapat dipersepsikan konflik kepentingan oleh publik o Menghindari rangkap jabatan o Membatasi dan mengurangi kepentingan pribadi yang dapat mempengaruhi pelaksanaan tugas o Mendorong perbaikan sistem pengelolaan konflik kepentingan di instansi untuk menutup celah pelanggaran terhadap aturan kebijakan konflik kepentingan Jikapun terjadi konflik kepentingan yang dilakukan antara lain o Melaporkan konflik kepentingan yang dihadapi sesuai dengan ketentuan dan proses yang berlaku di organisasi o Memberitahukan kepada atasan sebelum bertindak jika terdapat risiko terjadinya konflik kepentingan o Membatasi akses informasi atau menarik diri dari kegiatan yang sedang berlangsung o Menghindari pengambilan keputusan,3,"conflicts of interest arise when there is a conflict between the performance of duties and personal interests which influence decisions to be made and if this is not handled properly it can lead to violations of administrative ethics and even criminal penalties. therefore, in dealing with conflicts of interest at this time, what is being done is trying to prevent by avoiding or limiting sources of conflict of interest including o avoiding doing work outside of your current job o limiting or avoiding ownership of assets in the company that could be related to the implementation of job duties o avoiding and limiting yourself from interacting directly with related parties which is prohibited by the code of ethics and legislation or have the potential to be perceived as a conflict of interest by the public o avoid holding multiple positions o limit and reduce personal interests that can influence the implementation of duties o encourage improvements to the conflict of interest management system in the agency to close gaps in violations of conflict of interest policy rules if there is a conflict of interest between other o reporting conflicts of interest encountered in accordance with the provisions and processes applicable in the organization o notifying superiors before acting if there is a risk of a conflict of interest o limiting access to information or withdrawing from ongoing activities o avoiding decision making",conflicts of interest arise when there is a conflict between the performance of duties and personal interests.
1,AM,Untuk mewujudkan organisasi yang kuat maka setiap anggotanya wajib mematuhi segala aturan dan etika yang ada di organisasi tersebut Dengan mematuhinya maka agenda organisasi akan berjalan dengan lancar dan sesuai target sesuai program kerja karena setiap anggotanya berupaya untuk menciptakan suasana yang kondusif di dalam organisasi Hal hal yang dilakukan dalam upaya lebih mengoptimalkan penerapan kepatuhan terhadap aturan dan norma di antara rekan kerja atau organisasi diantaranya o saling menghargai dan memberi rasa hormat o berkomunikasi secara terbuka o saling memberikan contoh yang baik o saling memberikan kepercayaan o mendorong kepada seluruh pegawai untuk senantiasa mengikuti pelatihan yang berkualitas,3,"to create a strong organization, each member is obliged to comply with all the rules and ethics that exist in the organization. by complying with them, the organizat

In [8]:
# %pip install googletrans==4.0.0-rc1

from googletrans import Translator
translator = Translator()

def translate_text(text):
    result = translator.translate(text, src='id', dest='en')
    return result.text

In [9]:
# df['translated'] = df['answer'].apply(translate_text)
df['LEVEL KOMPETENSI'] = df['LEVEL KOMPETENSI'].astype(str)

## TRAIN/TEST SPLIT

In [10]:
# yahoo_answers_qa = yahoo_answers_qa["train"].train_test_split(test_size=0.3)
from sklearn.model_selection import train_test_split

# Membagi dataset menjadi data train (70%) dan data test (30%)
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

df_train

,INISIAL,RESPON,LEVEL KOMPETENSI,English,Summaries
8,DIS,Penerapan kepatuhan terhadap aturan dan norman rekan kerja atau organisasi terkadang kita juga merasa dalam penerapan kepatuhan terhadap aturan baik rekan kerja atau organisasi terkadang kita juga merasa dalam penerapan keputusan terhadap aturan baik rekan kerja dan organisasi adakalnya kita di buat dileman antara rekan kerja dan kita hadapi tetap berkomunikasi dengan rekan kerja atau diskusi dengan rekan kerja artinya kita dalam menerapkan aturan tersebut harus bisa kita bedakan mana yang dinamakan dengan aturan mana yang dapat kita kerjakan sebagai rekan kerja karena apabila kita tidak mengkordinasikan nya itu akan salah penerapan dan akan berdampak terhadap kinerja dan loyalitas kerja,1,"implementing compliance with the rules and norms of co-workers or the organization, sometimes we also feel that in implementing compliance with the rules of both co-workers or the organization, sometimes we also feel that in implementing decisions against the rules of both co-workers and the organization, sometimes we are put in a dilemma between co-workers and we still face it. communicating with co-workers or discussing with co-workers means that when we apply these rules we have to be able to distinguish which rules are called which ones we can work with as co-workers because if we don't coordinate them it will be misapplied and will have an impact on work performance and loyalty.",implementing compliance with the rules and norms of co-workers or the organization.
6,DFYM,Sering terjadi di lingkungan kerja khususnya di tempat kerja yang saya duduki ketika saya melihat rekan kerja di satu institusi masih ada hal hal yang secara aturan dan norma belum optimal baik terkait disiplin kinerja ataupun perilaku personal yang tidak membuat nyaman di lingkungan kerja Sikap yang lakukan Contoh Masalah kesatu 1 terkait dispilin kinerja dalam penyelesaian pertanggungjawaban kegiatan yang ditunda tunda bagi saya hal tersebut adalah suatu yang sangat mengganggu terhadap tata kelola administrasi Dalam kasus ini saya menyikapi nya sebagai sekretaris melakukan pendekatan pendekatan personal terlebih dahulu untuk mencari informasi kenapa dan alasan apa yang menyebabkan hal tersebut terjadi Selanjutnya saya memberikan arahan dan mencari solusi penyelasainnya ketika ada kesulitan dalam pertanggungjawaban tersebut Contoh Masalah kedua 2 terkait norma perilaku personal pegawai yang secara adab menyebabkan gangguan terhadap lingkungan kerja misalnya pegawai yang tidak ada sopan santun terhadap rekan kerja Hal yang sikapi adalah saya berusaha menjadi tauladan bagi lingkungan kerja saya mencoba untuk tidak memandang siapapun pegawai tersebut untuk masuk di tengah tengah pergaualan nya Saya membawa misi untuk merubah karakter buruk seseorang dengan pola dari diri saya terlebih dahulu untuk bisa memberikan energi positif bagi lingkungan kerja,2,"it often happens in the work environment, especially in the workplace that i occupy, when i see that colleagues at one institution still have things that are not optimal according to the rules and norms, whether related to performance discipline or personal behavior that does not make them comfortable in the work environment. attitudes that occur. examples of problems. the first 1 is related to performance discipline in resolving accountability for postponed activities. for me, this is something that is very disturbing to administrative governance. in this case, i responded as a secretary, taking a personal approach first to find information about why and what reasons caused this. happens. next, i give directions and look for solutions when there are difficulties in accountability. example of the second problem 2 related to employee personal behavior norms which in a civil manner cause disruption to the work environment, for example employees who have no manners towards colleagues. the thing that is addressed is that i try to be a role model. for the work environment, 

In [12]:
from datasets import DatasetDict, Dataset

# Misalkan X_train dan X_test adalah DataFrame dengan kolom 'id', 'answer', 'label', 'translated'
# Pilih kolom-kolom yang ingin Anda gunakan

# Ubah DataFrame menjadi dataset dengan menggunakan Dataset dari Hugging Face's transformers
train_dataset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

# Membuat DatasetDict dengan format yang diinginkan
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
})

# Menghapus kolom '__index_level_0__' dari "train" dataset
dataset["train"] = dataset["train"].remove_columns('__index_level_0__')

# Menghapus kolom '__index_level_0__' dari "test" dataset
dataset["test"] = dataset["test"].remove_columns('__index_level_0__')

# Menampilkan informasi dataset
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['INISIAL ', 'RESPON', 'LEVEL KOMPETENSI', 'English', 'Summaries'],
        num_rows: 43
    })
    test: Dataset({
        features: ['INISIAL ', 'RESPON', 'LEVEL KOMPETENSI', 'English', 'Summaries'],
        num_rows: 19
    })
})


## LOAD MODEL

In [13]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

## TOKENIZE

In [14]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["RESPON"], truncation=True), batched=True, remove_columns=['RESPON', 'LEVEL KOMPETENSI'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["LEVEL KOMPETENSI"], truncation=True), batched=True, remove_columns=['RESPON', 'LEVEL KOMPETENSI'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Max target length: 2


## PREPARED TRAIN

In [15]:
def preprocess_function(sample, padding="max_length"):
    # add prefix to the input for t5
    # inputs = [item for item in sample["RESPON"]]
    inputs = "IN;Penerapan kepatuhan terhadap aturan dan norman rekan kerja atau organisasi terkadang kita juga merasa dalam penerapan kepatuhan terhadap aturan baik rekan kerja atau organisasi terkadang kita juga merasa dalam penerapan keputusan terhadap aturan baik rekan kerja dan organisasi adakalnya kita di buat dileman antara rekan kerja dan kita hadapi tetap berkomunikasi dengan rekan kerja atau diskusi dengan rekan kerja artinya kita dalam menerapkan aturan tersebut harus bisa kita bedakan mana yang dinamakan dengan aturan mana yang dapat kita kerjakan sebagai rekan kerja karena apabila kita tidak mengkordinasikan nya itu akan salah penerapan dan akan berdampak terhadap kinerja dan loyalitas kerja"

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["LEVEL KOMPETENSI"], max_length=max_target_length, padding=padding, truncation=True)
    
    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['RESPON', 'LEVEL KOMPETENSI'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['INISIAL ', 'English', 'Summaries', 'input_ids', 'attention_mask', 'labels']


## Fine-tune and evaluate FLAN-T5

In [16]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [17]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("f1")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, average='macro')
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\harto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [20]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Define local output directory
local_output_dir = "D:/Analisis Psikotes/model_id"  # Ganti dengan path direktori lokal yang diinginkan

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=local_output_dir,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,  # Overflows with fp16
    learning_rate=3e-4,

    num_train_epochs=2,
    # logging & evaluation strategies
    logging_dir=f"{local_output_dir}/logs",
    logging_strategy="epoch",
    evaluation_strategy="no",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,
    report_to="tensorboard",
    push_to_hub=False,  # Tidak push ke Hugging Face Hub
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)


In [21]:
# Start training 
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 6.00 GiB total capacity; 5.24 GiB already allocated; 0 bytes free; 5.32 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [21]:
trainer.evaluate()

D:\myenv\Lib\site-packages\transformers\generation\utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 0.5575372576713562,
 'eval_f1': 31.6667,
 'eval_gen_len': 2.0,
 'eval_runtime': 37.2362,
 'eval_samples_per_second': 0.51,
 'eval_steps_per_second': 0.081,
 'epoch': 2.0}

In [22]:
# Save the trained model and tokenizer locally
local_model_dir = "D:/Analisis Psikotes/ourmodel"
model.save_pretrained(local_model_dir)
tokenizer.save_pretrained(local_model_dir)

('D:/Analisis Psikotes/ourmodel\\tokenizer_config.json',
 'D:/Analisis Psikotes/ourmodel\\special_tokens_map.json',
 'D:/Analisis Psikotes/ourmodel\\spiece.model',
 'D:/Analisis Psikotes/ourmodel\\added_tokens.json',
 'D:/Analisis Psikotes/ourmodel\\tokenizer.json')

## Run Inference and Classification Report

In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = '0'


In [24]:
from tqdm.auto import tqdm

samples_number = len(dataset['test'])
progress_bar = tqdm(range(samples_number))
predictions_list = []
labels_list = []
for i in range(samples_number):
  text = dataset['test']['RESPON'][i]
  inputs = tokenizer.encode_plus(text, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
  outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150, num_beams=4, early_stopping=True)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
  predictions_list.append(prediction)
  labels_list.append(dataset['test']['LEVEL KOMPETENSI'][i])

  progress_bar.update(1)

  0%|          | 0/19 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 5.24 GiB already allocated; 0 bytes free; 5.32 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF